This is file runs the main calculation for the flexible length TPS simulation. It requires the file `alanine_dipeptide_tps_equil.nc`, which is written in the notebook `alanine_dipeptide_tps_first_traj.ipynb`.

In this file, you will learn:
* how to set up and run a flexible length TPS simulation

NB: This is a long calculation. In practice, it would be best to export the Python from this notebook, remove the `live_visualizer`, and run non-interactively on a computing node.

In [1]:
from __future__ import print_function
%matplotlib inline
import openpathsampling as paths

In [2]:
from openpathsampling.engines import gromacs as ops_gmx

## Load engine, trajectory, and states from file

In [3]:
old_storage = paths.Storage("alanine_dipeptide_tps_equil.nc", mode='r')

In [4]:
options = {
    'gmx_executable': 'gmx -nobackup ',
    'snapshot_timestep': 0.02
}
# we create a new engine because we want to save in new directories
# (prod instead of equil)
engine = ops_gmx.Engine(gro="conf.gro",
                        mdp="md.mdp",
                        top="topol.top",
                        options=options,
                        base_dir=".",
                        prefix="prod").named("production")

# for these, we just load the ones we used to create the 
C_7eq = old_storage.volumes['C_7eq']
alpha_R = old_storage.volumes['alpha_R']
final_step = old_storage.steps[-1]
traj = final_step.active[0].trajectory
phi = old_storage.cvs['phi']
psi = old_storage.cvs['psi']

template = traj[0]  # any snapshot is fine

In [5]:
# make sure we store the calculated versions of phi and psi
phi.enable_diskcache()
psi.enable_diskcache()

In [6]:
print(engine.name)

production


## TPS

As always, the process for setting up a simulation is:

1. Create a `network`
2. Create a `move_scheme`
3. Set up `initial_conditions`
4. Create the `PathSampling` object and run it.

Since we already created all the input to these when we set up the first trajectory, we can use the versions we loaded above.

In [7]:
network = paths.TPSNetwork(C_7eq, alpha_R)

In [8]:
scheme = paths.OneWayShootingMoveScheme(network, selector=paths.UniformSelector(), engine=engine)

In [9]:
initial_conditions = scheme.initial_conditions_from_trajectories(traj)

No missing ensembles.
No extra ensembles.


In [10]:
storage = paths.Storage("alanine_dipeptide_tps.nc", mode="w")
storage.save(template)  # required for diskcache
sampler = paths.PathSampling(storage=storage,
                             move_scheme=scheme,
                             sample_set=initial_conditions)

In [11]:
# This sets up logging, to provide additional information about what is happening

import logging
import sys

# use logging.INFO for basic info; logging.DEBUG for details
log_level = logging.INFO

root = logging.getLogger('openpathsampling.engines')
root.setLevel(logging.DEBUG)  # let the handlers sort it out
ch = logging.StreamHandler(sys.stdout)
ch.setLevel(log_level)
root.addHandler(ch)

# uncomment the following to write detailed debug info to a file
#fh = logging.FileHandler('debug_details.log')
#fh.setLevel(logging.DEBUG)
#root.addHandler(fh)

Note: 1000 steps will take a long time. If you just want to run a little bit, reduce this number.

In [12]:
#sampler.live_visualizer = paths.StepVisualization2D(network, phi, psi, [-3.14, 3.14], [-3.14, 3.14])
sampler.run(1000)

Working on Monte Carlo cycle number 10
Running for 2 minutes 36 seconds - 17.39 seconds per step
Estimated time remaining: 17 seconds
Starting trajectory
gmx -nobackup grompp -c ./conf.gro -f ./md.mdp -p ./topol.top -t ./initial_frame.trr 
gmx -nobackup mdrun -s topol.tpr -o ./prod_trr/0000010.trr -e ./prod_edr/0000010.edr -g ./prod_log/0000010.log 
Started engine: psutil.Popen(pid=71684, name='gmx', started='15:07:35')
Through frame: 0
Through frame: 10
total_time 6.7042
About to send signal Signals.SIGTERM to psutil.Popen(pid=71684, name='gmx', started='15:07:35')
Finished trajectory, length: 21
DONE! Completed 10 Monte Carlo cycles.


In [13]:
# IF YOU HAVE AN ERROR: uncomment and run
# engine.stop(None)
# (this will stop the Gromacs process)

In [14]:
# IF YOU RE-RUN, DELETE EXISTING FILES AND RESTART THE NOTEBOOK
#!rm -rf prod_trr prod_log prod_edr

With this done, you can go on to do the flexible-length parts of the analysis in `alanine_dipeptide_tps_analysis.ipynb`.

In [15]:
storage.close()